Original Dolly: https://www.databricks.com/blog/2023/03/24/hello-dolly-democratizing-magic-chatgpt-open-models.html

My Dolly: https://huggingface.co/Bandika/Dollyka

Cleaned Alpaca Dataset: https://github.com/gururise/AlpacaDataCleaned

In [1]:
!pip -q install datasets loralib sentencepiece
!pip -q install git+https://github.com/huggingface/transformers
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.7 MB/s eta 0:00:

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from peft import PeftModel
from transformers import AutoTokenizer, GPTJForCausalLM, GenerationConfig

model = GPTJForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6B",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "Bandika/Dollyka")

In [4]:
def dolly_generate(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )

    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=128,
        pad_token_id = 0,
        eos_token_id = 50256
    )

    for s in generation_output.sequences:
        print(tokenizer.decode(s))

In [5]:
%%time
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Write a python program that calculates Fibonacci sequences.
### Response:"""

dolly_generate(PROMPT)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

Generating...
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Write a python program that calculates Fibonacci sequences.
### Response:
The Python programming language provides many built-in functions for performing mathematical operations, such as addition and subtraction. The fibonacci function can be used to calculate any sequence of numbers in which each number after the first two are equal to the sum of those before it (e.g., 1 + 2 = 3). Here's how you could use this function to compute the Fibonacci series:
```python
def fib(n):  # n represents the starting value; here we'll start with 0
    if n == 0 or n == 1: return n

    else: return fib
CPU times: user 30.4 s, sys: 727 ms, total: 31.1 s
Wall time: 37.1 s


In [6]:
%%time
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What is the difference between Hungary and Austria?
### Response:"""

dolly_generate(PROMPT)

Generating...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What is the difference between Hungary and Austria?
### Response:
The two countries are located in Central Europe, with both being part of the European Union (EU). They share many similarities such as their language, culture, history, economy, etc., but there are also some differences between them. For example, while they have similar cultures, Hungarian cuisine tends to be more spicy than Austrian food due to its proximity to India. In addition, Austrians tend to speak German whereas Hungarians prefer speaking Hungarian. The main difference between these two nations lies within their political systems; while both are parliamentary democracies, Hungary has been governed by Fidesz since 1989, whereas Austria was ruled by the Social Democratic Party
CPU times: user 26.7 s, sys: 0 ns, total: 26.7 s
Wall time: 26.7 s
